In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import warnings
warnings.filterwarnings("ignore")
from datetime import datetime
import datetime as dt
from datetime import timedelta

import plotly.colors
import plotly.express as px
import plotly.graph_objects as go

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/ecommerce-data/data.csv


In [3]:
ecommerce_data = pd.read_csv('/kaggle/input/ecommerce-data/data.csv',encoding='unicode_escape')
ecommerce_data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [4]:
ecommerce_data.shape

(541909, 8)

In [5]:
ecommerce_data.isna().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

In [6]:
ecommerce_data[ecommerce_data.CustomerID.isnull()][ecommerce_data.InvoiceNo=='536544']

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
1443,536544,21773,DECORATIVE ROSE BATHROOM BOTTLE,1,12/1/2010 14:32,2.51,NaN,United Kingdom
1444,536544,21774,DECORATIVE CATS BATHROOM BOTTLE,2,12/1/2010 14:32,2.51,NaN,United Kingdom
1445,536544,21786,POLKADOT RAIN HAT,4,12/1/2010 14:32,0.85,NaN,United Kingdom
1446,536544,21787,RAIN PONCHO RETROSPOT,2,12/1/2010 14:32,1.66,NaN,United Kingdom
1447,536544,21790,VINTAGE SNAP CARDS,9,12/1/2010 14:32,1.66,NaN,United Kingdom
...,...,...,...,...,...,...,...,...
1965,536544,21721,CANDY SHOP STICKER SHEET,1,12/1/2010 14:32,1.66,NaN,United Kingdom
1966,536544,21722,SWEET PUDDING STICKER SHEET,1,12/1/2010 14:32,1.66,NaN,United Kingdom
1967,536544,21731,RED TOADSTOOL LED NIGHT LIGHT,5,12/1/2010 14:32,3.36,NaN,United Kingdom
1968,536544,21742,LARGE ROUND WICKER PLATTER,1,12/1/2010 14:32,12.72,NaN,United Kingdom


### For now, we will drop CustomerID rows which are null. However, since this will remove 30% of the data, as a next iteration, we should try replacing null values of CustomerID based on the InvoiceNo, since there are no null values in this.

In [7]:
ecommerce_data.dropna(subset=['CustomerID'], inplace=True)

In [8]:
ecommerce_data['TotalAmount'] = ecommerce_data['Quantity']*ecommerce_data['UnitPrice']
ecommerce_data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalAmount
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34


In [9]:
ecommerce_data.dtypes

InvoiceNo       object
StockCode       object
Description     object
Quantity         int64
InvoiceDate     object
UnitPrice      float64
CustomerID     float64
Country         object
TotalAmount    float64
dtype: object

In [10]:
ecommerce_data['InvoiceDate'] = pd.to_datetime(ecommerce_data['InvoiceDate'])

### Since this is an old dataset, we need to add a reference date to reduce the Recency value. Keeping the original date will have the Recency value too large to be true or relevant. 

In [11]:
reference_date = pd.Timestamp(datetime.now().date())
reference_date

Timestamp('2024-08-29 00:00:00')

In [12]:
ecommerce_data['InvoiceDate'].max()

Timestamp('2011-12-09 12:50:00')

In [13]:
timedelta(days=1)

datetime.timedelta(days=1)

In [14]:
reference_date = ecommerce_data['InvoiceDate'].max() + timedelta(days=1)
reference_date

Timestamp('2011-12-10 12:50:00')

### We will now use this reference date to calculate Recency, to find relevant groups as of 2011-12-10

### Calculate RFM - Recency, Frequency and Monetary Value

In [15]:
rfm = ecommerce_data.groupby('CustomerID').agg({
    'InvoiceDate': lambda x: (reference_date - x.max()).days,
    'InvoiceNo': 'count',
    'TotalAmount': 'sum' 
})

rfm.head()

,InvoiceDate,InvoiceNo,TotalAmount
CustomerID,,,
12346.0,326,2,0.00
12347.0,2,182,4310.00
12348.0,75,31,1797.24
12349.0,19,73,1757.55
12350.0,310,17,334.40


In [16]:
rfm.rename(columns={'InvoiceDate': 'Recency', 'InvoiceNo': 'Frequency', 'TotalAmount': 'MonetaryValue'}, inplace=True)
rfm.head()

,Recency,Frequency,MonetaryValue
CustomerID,,,
12346.0,326,2,0.00
12347.0,2,182,4310.00
12348.0,75,31,1797.24
12349.0,19,73,1757.55
12350.0,310,17,334.40


In [17]:
quantiles = rfm.quantile(q=[0.25,0.5,0.75])
quantiles

,Recency,Frequency,MonetaryValue
0.25,17.0,17.0,293.3625
0.50,50.0,42.0,648.0750
0.75,143.0,102.0,1611.7250


In [21]:
#Assign RFM Score

def RScore(x,p,d):
    if p=='Recency':
        if x <= d[p][0.25]:
            return 4
        elif x <= d[p][0.50]:
            return 3
        if x <= d[p][0.75]:
            return 2
        else:
            return 1
    else:
        if x <= d[p][0.25]:
            return 1
        elif x <= d[p][0.50]:
            return 2
        if x <= d[p][0.75]:
            return 3
        else:
            return 4  
        
   

In [27]:
rfm['R'] =  rfm['Recency'].apply(RScore, args=('Recency',quantiles,))
rfm['F'] =  rfm['Frequency'].apply(RScore, args=('Frequency',quantiles,))      
rfm['M'] =  rfm['MonetaryValue'].apply(RScore, args=('MonetaryValue',quantiles,))   
rfm.head()

,Recency,Frequency,MonetaryValue,R,F,M
CustomerID,,,,,,
12346.0,326,2,0.00,1,1,1
12347.0,2,182,4310.00,4,4,4
12348.0,75,31,1797.24,2,2,4
12349.0,19,73,1757.55,3,3,4
12350.0,310,17,334.40,1,1,2


In [29]:
rfm['RFM_segment'] = rfm['R'].astype(str) + rfm['F'].astype(str) + rfm['M'].astype(str)
rfm.head()

,Recency,Frequency,MonetaryValue,R,F,M,RFM_segment
CustomerID,,,,,,,
12346.0,326,2,0.00,1,1,1,111
12347.0,2,182,4310.00,4,4,4,444
12348.0,75,31,1797.24,2,2,4,224
12349.0,19,73,1757.55,3,3,4,334
12350.0,310,17,334.40,1,1,2,112


In [30]:
rfm['RFM_Score'] = rfm[['R','F','M']].sum(axis=1)
rfm.head()

,Recency,Frequency,MonetaryValue,R,F,M,RFM_segment,RFM_Score
CustomerID,,,,,,,,
12346.0,326,2,0.00,1,1,1,111,3
12347.0,2,182,4310.00,4,4,4,444,12
12348.0,75,31,1797.24,2,2,4,224,8
12349.0,19,73,1757.55,3,3,4,334,10
12350.0,310,17,334.40,1,1,2,112,4


In [32]:
segment_label = ['Low-Value','Mid-Value','High-Value']

def assign_segment(score):
    if score < 5:
        return segment_label[0]
    elif score < 9:
        return segment_label[1]
    else:
        return segment_label[2]

rfm['RFM_Segment_Label'] = rfm['RFM_Score'].apply(assign_segment)
rfm.head()

,Recency,Frequency,MonetaryValue,R,F,M,RFM_segment,RFM_Score,RFM_Segment_Label
CustomerID,,,,,,,,,
12346.0,326,2,0.00,1,1,1,111,3,Low-Value
12347.0,2,182,4310.00,4,4,4,444,12,High-Value
12348.0,75,31,1797.24,2,2,4,224,8,Mid-Value
12349.0,19,73,1757.55,3,3,4,334,10,High-Value
12350.0,310,17,334.40,1,1,2,112,4,Low-Value


In [41]:
segment_counts = rfm['RFM_Segment_Label'].value_counts().reset_index()
segment_counts.columns = ['RFM_Segment','Count']
segment_counts.sort_values('RFM_Segment')

,RFM_Segment,Count
1,High-Value,1690
2,Low-Value,783
0,Mid-Value,1899


In [47]:
fig = px.bar(segment_counts, x='RFM_Segment', y='Count',
            title="Customer distribution by RFM Segment", color='RFM_Segment', color_discrete_sequence=px.colors.qualitative.Pastel,
            labels={'RFM_Segment': 'RFM_Segment', 'Count': 'No of Customers'})
fig.show()